<a href="https://colab.research.google.com/github/jansoe/dl_workshop/blob/main/Squad_wt_Transformers_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pretrained SQuAD Transformer


In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.5MB 9.6MB/s 
     |████████████████████████████████| 3.3MB 48.0MB/s 
     |████████████████████████████████| 901kB 47.8MB/s 
     |████████████████████████████████| 245kB 7.7MB/s 
     |████████████████████████████████| 122kB 26.8MB/s 
     |████████████████████████████████| 245kB 20.2MB/s 


In [2]:
from pprint import pprint
import os
import torch
import torch.utils.data as datatools
import transformers

Sollten Sie Interesse daran haben, welche Hardware Ihnen von Google zur Verfügung gestellt wird, können Sie die folgende Code-Zelle ausführen.

In [3]:
torch.cuda.get_device_name()

'Tesla T4'

##  Model

In [7]:
# model_name_or_path = 'distilbert-base-uncased-distilled-squad' 
# model_name_or_path = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model_name_or_path = 'deepset/gelectra-base-germanquad'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)
model = transformers.AutoModelForQuestionAnswering.from_pretrained(model_name_or_path)

In [8]:
tokenizer.tokenize('This is a test of the tokenizer')

['This', 'is', 'a', 'test', 'of', 'the', 'to', '##ken', '##izer']

In [11]:
tokenizer.tokenize('Was passiert mit einem deutschen Satz und einer Wortzusammensetzung')

['Was',
 'passiert',
 'mit',
 'einem',
 'deutschen',
 'Satz',
 'und',
 'einer',
 'Wort',
 '##zusammen',
 '##setzung']

##  Inference

In [15]:
context = """\
The Rhine (Romansh: Rein, German: Rhein, French: le Rhin, Dutch: Rijn) \
is a European river that begins in the Swiss canton of Graubünden in the southeastern Swiss Alps, \
forms part of the Swiss-Austrian, Swiss-Liechtenstein border, Swiss-German and then the Franco-German border, \
then flows through the Rhineland and eventually empties into the North Sea in the Netherlands. \
The biggest city on the river Rhine is Cologne, Germany with a population of more than 1,050,000 people. \
It is the second-longest river in Central and Western Europe (after the Danube), at about 1,230 km (760 mi),\
[note 2][note 1] with an average discharge of about 2,900 m3/s (100,000 cu ft/s).\
"""

question =  "Where does the Rhine begin?"
# question = "What is the largest city the Rhine runs through? "
# question = "What river is larger than the Rhine?"

#### Tokenize input

In [16]:
inputs = tokenizer.encode_plus(
    question, 
    context, 
    return_attention_mask = False, 
    return_tensors = 'pt',
    return_token_type_ids = not('distilbert' in model_name_or_path) 
)

inputs

{'input_ids': tensor([[  102,  5376,  1056,  5601,   115,   871, 11025,   844,   972,   113,
          1992,   103,   996, 11025,   844,   201, 27461, 30889,   853,  4169,
           818,  5019,   853,  3774,   818, 24917,   853,  1997, 11025,   113,
           818,   840, 24577, 30889,   853, 11078, 15116,  2530,  1420,   120,
         11225,  9710,   289,  6711,   972,  2234,   153,   871, 20545,  9051,
          4659,   917, 30567,  4178, 21081, 30882,   153,   871,   262,  3212,
         12032,  7698, 20545, 12706, 30886,   818,  2801, 30886, 10818,   917,
           871, 20545,   232, 17313, 30882,   818, 20545,   232, 21177,  7732,
          7632,   105,   818, 20545,   232,  5019,  1257,   871, 30882,   871,
         24810,   232,  5019,  7732,  7632,   105,   818,   871, 30882, 13991,
          6186, 16453, 18183,   871, 11025,   844,   492,  1257, 10172,  7785,
          2241, 17400,   185,   115, 26378,   871,  9536, 25143,   153,   871,
          8878,   452,  4690,   566,  

#### Model predictions

In [17]:
out = model(**inputs)
out

QuestionAnsweringModelOutput([('start_logits',
                               tensor([[-4.1477, -4.3796, -5.3886, -5.8432, -6.2011, -5.7395, -6.1887, -6.5280,
                                        -5.9422, -6.6591, -5.3750, -4.1477, -1.4374, -4.7159, -3.6722, -2.9581,
                                        -5.4065, -6.1306, -5.9610, -5.4363, -5.8917, -5.3823, -5.7580, -5.2957,
                                        -6.0031, -5.6724, -6.1080, -5.3913, -5.8815, -6.1443, -5.7425, -5.3519,
                                        -6.0029, -6.0605, -6.1094, -5.7092, -6.3144, -2.6752, -1.7434, -2.0534,
                                        -1.9726, -4.5848, -3.9979,  0.7013,  1.0322,  0.9215,  6.2602,  2.1015,
                                         0.8172, -2.5703, -3.4227, -4.5868, -3.5444, -5.5673, -5.2932, -2.9465,
                                        -2.6179, -4.5194, -3.7431, -5.2662, -5.5091, -5.4214, -4.3388, -5.4418,
                                        -4.8818, -5.4307,

#### Convert token_id to tokens

In [19]:
all_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].numpy().squeeze())
#all_tokens

#### Take highet rated answer string

In [20]:
answer = ' '.join(all_tokens[torch.argmax(out.start_logits) : torch.argmax(out.end_logits)+1])
answer

'in the Swiss can ##ton of Grau ##bü ##nde ##n'

In [21]:
answer = answer.replace(' ##', '')
answer

'in the Swiss canton of Graubünden'

### Evaluation function

In [ ]:
def evaluate(model, context, question):
    inputs = tokenizer.encode_plus(
        question, 
        context, 
        return_attention_mask = False, 
        return_tensors = 'pt',
        return_token_type_ids = not('distilbert' in model_name_or_path) 
    )

    # prediction for every token if it is start or end token
    out = model(**inputs)

    # we convert tokens to ids
    all_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].numpy().squeeze())

    # and combine all tokens from start to end
    answer = ' '.join(all_tokens[torch.argmax(out.start_logits) : torch.argmax(out.end_logits)+1])
    answer = answer.replace(' ##', '')
    return answer

In [ ]:
context = """\
The Rhine (Romansh: Rein, German: Rhein, French: le Rhin, Dutch: Rijn) \
is a European river that begins in the Swiss canton of Graubünden in the southeastern Swiss Alps, \
forms part of the Swiss-Austrian, Swiss-Liechtenstein border, Swiss-German and then the Franco-German border, \
then flows through the Rhineland and eventually empties into the North Sea in the Netherlands. \
The biggest city on the river Rhine is Cologne, Germany with a population of more than 1,050,000 people. \
It is the second-longest river in Central and Western Europe (after the Danube), at about 1,230 km (760 mi),\
[note 2][note 1] with an average discharge of about 2,900 m3/s (100,000 cu ft/s).\
"""

question =  "Where does the Rhine begin?"
# question = "What is the largest city the Rhine runs through? "
# question = "What river is larger than the Rhine?"

evaluate(model=model, context=context, question=question)

'swiss canton of graubunden'

## Task
Teste verschiedene Modelle mit witeren Kontexten und Fragen